In [1]:
import numpy as np
from hyperopt import hp, fmin, tpe, space_eval
from scikits.odes.ode import ode
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
data = pd.read_csv('equations_static.csv')
data

,Reaction Name,Reactant(s),Product(s),Reaction Rate
0,NaN,NaN,NaN,NaN
1,PA1_Chemical_Dilution:,PA1_Chemical,NaN,PA1_Chemical * PA1_Chemical_dilution_rate
2,PA2_Chemical_Dilution:,PA2_Chemical,NaN,PA2_Chemical * PA2_Chemical_dilution_rate
3,PA3_Chemical_Dilution:,PA3_Chemical,NaN,PA3_Chemical * PA3_Chemical_dilution_rate
4,PAF_Chemical_Dilution:,PAF_Chemical,NaN,PAF_Chemical * PAF_Chemical_dilution_rate
...,...,...,...,...
57,PapB_Protein_Folded_Catalyzed_Biosynthesis:,PA1_Chemical + PapB_Protein_Folded,PA2_Chemical + PapB_Protein_Folded,PapB_Protein_Folded_kcat * ((PA1_Chemical / Av...
58,PapC_Protein_Folded_Catalyzed_Biosynthesis:,PapC_Protein_Folded + PA2_Chemical,PapC_Protein_Folded + PA3_Chemical,PapC_Protein_Folded_kcat * PapC_Protein_Folded...
59,Deaminase_Catalyzed_Biosynthesis:,Deaminase + PA3_Chemical,Deaminase + PAF_Chemical,Deaminase_kcat * Deaminase * ((PA3_Chemical / ...
60,LAAO_Protein_Folded_Catalyzed_Biosynthesis:,LAAO_Protein_Folded + PAF_Chemical,LAAO_Protein_Folded + PACA_Chemical_Internal,LAAO_Protein_Folded_kcat * LAAO_Protein_Folded...


In [7]:
data['Reactant(s)'].unique()

array([nan, 'PA1_Chemical', 'PA2_Chemical', 'PA3_Chemical',
       'PAF_Chemical', 'PACA_Chemical_Internal', 'Chorismate_Chemical',
       'Deaminase', 'Promoter1', 'Promoter2', 'Promoter3',
       'Pap_Operon_mRNA', 'PapA_Protein_Unfolded', 'PapA_Protein_Folded',
       'PapB_Protein_Unfolded', 'PapB_Protein_Folded',
       'PapC_Protein_Unfolded', 'PapC_Protein_Folded', 'LAAO_mRNA',
       'LAAO_Protein_Unfolded', 'LAAO_Protein_Folded', 'Efflux_Pump_mRNA',
       'Efflux_Pump_Protein_Unfolded', 'Efflux_Pump_Protein_Folded',
       'PapA_Protein_Folded + Chorismate_Chemical',
       'PA1_Chemical + PapB_Protein_Folded',
       'PapC_Protein_Folded + PA2_Chemical', 'Deaminase + PA3_Chemical',
       'LAAO_Protein_Folded + PAF_Chemical',
       'Efflux_Pump_Protein_Folded + PACA_Chemical_Internal'],
      dtype=object)

In [15]:
(5E1 + 5E2)/2

275.0

In [16]:
def p_aminostyrene(t, y, ydot, params):
    chorismate, pa1, pa2, pa3, paf, paca_int, paca_ext, promoter1, pap_mrna, pap_ared_folded, pap_ared_cleaved, papA_uf, papA, papB_uf, papB, papC_uf, papC, deaminase, promoter2, laao_mrna, laao_ared_folded, laao_ared_cleaved, laao_uf, laao, promoter3, eff_mrna, eff_ared_folded, eff_ared_cleaved, eff_uf, eff = y

    A, W = params

    dilution_rate = 5.79E-4
    dna_duplication_rate = 5.78E-4
    ki = 5E-5 #range [1E-5, 1E-4]
    tl = 50 #range [1E1, 1E2]
    tp = 50 #range [1E1, 1E2]
    ta = 5E-4 #range [1E-4, 1E-3]
    toxicity_factor = ki/(ki + (ta*paca_int + tp*ef + tl*laao))

    pap_mrnalength = 3400
    eff_mrnalength = 2900
    laao_mrnalength = 1600
    tr_mrnalength = 1100
    srna_length = 200
    cell_volume = 2.5E-15
    ribosome_elongation = 20
    rnap_elongation = 25
    ligand_activation_ratio = 0.17
    txn_init = 2E-1
    tsn_init = 2E-1

    mrna_degradation_rate = 3E-3 #range [3E-4, 3E-2]
    protein_degradation_rate = 2E-4 #range [2E-5, 2E-3]
    cleaved_mrna_degradation_rate = 0.
    protein_folding_rate = 2E0 #range [2E-1, 2E1]
    pap_translation_rate = ((pap_mrna)/(tsn_init + (pap_mrnalength/ribosome_elongation))) 
    pap_ared_folded_translation_rate = ((pap_ared_folded)/(tsn_init + (pap_mrnalength/ribosome_elongation))) 
    pap_ared_cleaved_translation_rate = ((pap_ared_cleaved)/(tsn_init + (pap_mrnalength/ribosome_elongation))) 
    ared_folding_rate = 1.5E-2 #range [1.5E-3, 1.5E-1]
    kobs_m = 1E-2 #range [1E-4, 1E-1]
    EC50 = 1E-6 #range [1E-7, 1E-5]
    ligand_activation_ratio_cleavage = 1E1 #range [1E0, 1E2]
    kobs_p = kobs_m * ligand_activation_ratio_cleavage

    pap_ared_cleavage_rate = ((pap_ared_folded*(kobs_p - kobs_m))/(1+(EC50/ligand))) + kobs_m #Ligand should be determined by architecture?? FIX THIS
    laao_ared_cleavage_rate = pap_ared_cleavage_rate#FIX THIS
    eff_ared_cleavage_rate = pap_ared_cleavage_rate #FIX THIS

    laao_translation_rate = ((laao_mrna)/(tsn_init + (laao_mrnalength/ribosome_elongation))) 
    laao_ared_folded_translation_rate = ((laao_ared_folded)/(tsn_init + (laao_mrnalength/ribosome_elongation))) 
    laao_ared_cleaved_translation_rate = ((laao_ared_cleaved)/(tsn_init + (laao_mrnalength/ribosome_elongation))) 
    
    eff_translation_rate = ((eff_mrna)/(tsn_init + (eff_mrnalength/ribosome_elongation))) 
    eff_ared_folded_translation_rate = ((eff_ared_folded)/(tsn_init + (eff_mrnalength/ribosome_elongation))) 
    eff_ared_cleaved_translation_rate = ((eff_ared_cleaved)/(tsn_init + (eff_mrnalength/ribosome_elongation))) 
    
    deaminase_production_rate =  1E1 #range [1E0, 1E2]
    chorismate_production_rate = 1100. #range [2E2, 2E3]
    enzyme_kcat = 5E0 #range [5E-1, 5E1]
    enzyme_km = 1E-6 #range [1E-7, 1E-5]
    avogadro = 6.0221408e+23
    efflux_rate = 275. #range [5E1, 5E2]

    #Genetic equations - must be affected by control topology - FIX THIS
    pap_mrna_transcription_rate = ((promoter1)/(txn_init + (pap_mrnalength/rnap_elongation))) #Uncertain if this is correct
    laao_mrna_transcription_rate = pap_mrna_transcription_rate
    eff_mrna_transcription_rate = pap_mrna_transcription_rate

    #Write out all 92 reactions in the cell
    pa1_dil = pa1*dilution_rate
    pa2_dil = pa2*dilution_rate
    pa3_dil = pa3*dilution_rate
    paf_dil = paf*dilution_rate
    paca_int_dil = paca_int*dilution_rate
    cho_dil = chorismate*dilution_rate
    deam_dil = deaminase*dilution_rate
    paf_loss = paf*1.40E-5 #range [1.4E-6, 1.4E-4]
    prom1_div = promoter1*dna_duplication_rate
    prom1_dil = promoter1*dilution_rate
    prom2_div = promoter2*dna_duplication_rate
    prom2_dil = promoter2*dilution_rate
    prom3_div = promoter3*dna_duplication_rate
    prom3_dil = promoter3*dilution_rate
    pap_mrna_txn =  promoter1 * pap_mrna_transcription_rate * toxicity_factor #FIX THIS
    pap_mrna_dil = pap_mrna * dilution_rate
    pap_mrna_deg = pap_mrna * mrna_degradation_rate * toxicity_factor
    papA_tsn = pap_translation_rate * toxicity_factor
    papA_folding = papA_uf * protein_folding_rate * toxicity_factor
    papA_uf_dil = papA_uf* dilution_rate
    papA_dil = papA*dilution_rate
    papA_uf_deg = papA_uf * protein_degradation_rate * toxicity_factor
    papA_deg = papA * protein_degradation_rate * toxicity_factor
    papB_tsn = pap_translation_rate * toxicity_factor
    papB_folding = papB_uf * protein_folding_rate * toxicity_factor
    papB_uf_dil = papB_uf * dilution_rate
    papB_dil = papB * dilution_rate
    papB_uf_deg = papB_uf * protein_degradation_rate * toxicity_factor
    papB_deg = papB * protein_degradation_rate * toxicity_factor
    papC_tsn = pap_translation_rate * toxicity_factor
    papC_folding = papC_uf * protein_folding_rate * toxicity_factor
    papC_uf_dil = papC_uf * dilution_rate
    papC_dil = papC * dilution_rate
    papC_uf_deg = papC_uf * protein_degradation_rate * toxicity_factor
    papC_deg = papC * protein_degradation_rate * toxicity_factor
    papA_ared_folded_tsn = pap_ared_folded * pap_ared_folded_translation_rate * toxicity_factor
    papA_ared_cleaved_tsn = pap_ared_cleaved * pap_ared_cleaved_translation_rate * toxicity_factor
    papB_ared_folded_tsn = pap_ared_folded * pap_ared_folded_translation_rate * toxicity_factor
    papB_ared_cleaved_tsn = pap_ared_cleaved * pap_ared_cleaved_translation_rate * toxicity_factor
    papC_ared_folded_tsn = pap_ared_folded * pap_ared_folded_translation_rate * toxicity_factor
    papC_ared_cleaved_tsn = pap_ared_cleaved * pap_ared_cleaved_translation_rate * toxicity_factor
    pap_ared_folding = pap_mrna * ared_folding_rate * toxicity_factor
    pap_ared_cleaving = pap_ared_folded * pap_ared_cleavage_rate * toxicity_factor #Combined bound and unbound cleaving
    pap_ared_folded_dil = pap_ared_folded * dilution_rate
    pap_ared_cleaved_dil = pap_ared_cleaved * dilution_rate
    pap_ared_folded_deg = pap_ared_folded * mrna_degradation_rate
    pap_ared_cleaved_deg = pap_ared_cleaved * cleaved_mrna_degradation_rate
    laao_mrna_txn =  promoter2 * laao_mrna_transcription_rate * toxicity_factor #FIX THIS\
    laao_mrna_dil = laao_mrna * dilution_rate
    laao_mrna_deg = laao_mrna * mrna_degradation_rate * toxicity_factor
    laao_tsn = laao_mrna * laao_translation_rate * toxicity_factor
    laao_folding = laao_uf * protein_folding_rate * toxicity_factor
    laao_uf_dil = laao_uf * dilution_rate
    laao_dil = laao * dilution_rate
    laao_uf_deg = laao_uf * protein_degradation_rate * toxicity_factor
    laao_deg = laao * protein_degradation_rate * toxicity_factor
    laao_ared_folded_tsn = laao_ared_folded * laao_ared_folded_translation_rate * toxicity_factor
    laao_ared_cleaved_tsn = laao_ared_cleaved * laao_ared_cleaved_translation_rate * toxicity_factor
    laao_ared_folding = laao_mrna * ared_folding_rate * toxicity_factor
    laao_ared_cleaving = laao_ared_folded * laao_ared_cleavage_rate * toxicity_factor #Combined bound and unbound cleaving
    laao_ared_folded_dil = laao_ared_folded * dilution_rate
    laao_ared_cleaved_dil = laao_ared_cleaved * dilution_rate
    laao_ared_folded_deg = laao_ared_folded * mrna_degradation_rate
    laao_ared_cleaved_deg = laao_ared_cleaved * cleaved_mrna_degradation_rate
    eff_mrna_txn =  promoter3 * eff_mrna_transcription_rate * toxicity_factor #FIX THIS
    eff_mrna_dil = eff_mrna * dilution_rate
    eff_mrna_deg = eff_mrna * mrna_degradation_rate * toxicity_factor
    eff_tsn = eff_mrna * eff_translation_rate * toxicity_factor
    eff_folding = eff_uf * protein_folding_rate * toxicity_factor
    eff_uf_dil = eff_uf * dilution_rate
    eff_dil = eff * dilution_rate
    eff_uf_deg = eff_uf * protein_degradation_rate * toxicity_factor
    eff_deg = eff * protein_degradation_rate * toxicity_factor
    eff_ared_folded_tsn = eff_ared_folded * eff_ared_folded_translation_rate * toxicity_factor
    eff_ared_cleaved_tsn = eff_ared_cleaved * eff_ared_cleaved_translation_rate * toxicity_factor
    eff_ared_folding = eff_mrna * ared_folding_rate * toxicity_factor
    eff_ared_cleaving = eff_ared_folded * eff_ared_cleavage_rate * toxicity_factor #Combined bound and unbound cleaving
    eff_ared_folded_dil = eff_ared_folded * dilution_rate
    eff_ared_cleaved_dil = eff_ared_cleaved * dilution_rate
    eff_ared_folded_deg = eff_ared_folded * mrna_degradation_rate
    eff_ared_cleaved_deg = eff_ared_cleaved * cleaved_mrna_degradation_rate
    deaminase_biosynthesis = deaminase_production_rate * toxicity_factor
    chorismate_biosynthesis = chorismate_production_rate * toxicity_factor
    papA_catalyzed_biosynthesis = enzyme_kcat * papA * ((chorismate / avogadro) / cell_volume) / (enzyme_km + ((chorismate / avogadro) / cell_volume)) * toxicity_factor
    papB_catalyzed_biosynthesis = enzyme_kcat * ((pa1 / avogadro) / cell_volume) / (enzyme_km + ((pa1 / avogadro) / cell_volume)) * papB * toxicity_factor
    papC_catalyzed_biosynthesis = enzyme_kcat * papC * ((pa2 / avogadro) / cell_volume) / (enzyme_km + ((pa2 / avogadro) / cell_volume)) * toxicity_factor
    deaminase_catalyzed_biosynthesis = enzyme_kcat * deaminase * ((pa3 / avogadro) / cell_volume) / (enzyme_km + ((pa3 / avogadro) / cell_volume)) * toxicity_factor
    laao_catalyzed_biosynthesis = enzyme_kcat * laao * ((paf / avogadro) / cell_volume) / (enzyme_km + ((paf / avogadro) / cell_volume)) * toxicity_factor
    paca_external_efflux = eff * ((paca_int / avogadro) / cell_volume) * efflux_rate * toxicity_factor

    #Create differential equations from reactions above
    ydot[0] = chorismate_biosynthesis - papA_catalyzed_biosynthesis - cho_dil #chorismate
    ydot[1] = papA_catalyzed_biosynthesis - papB_catalyzed_biosynthesis - pa1_dil #pa1
    ydot[2] = papB_catalyzed_biosynthesis - papC_catalyzed_biosynthesis - pa2_dil #pa2
    ydot[3] = papC_catalyzed_biosynthesis - deaminase_catalyzed_biosynthesis - pa3_dil #pa3
    ydot[4] = deaminase_catalyzed_biosynthesis -  laao_catalyzed_biosynthesis - paf_dil - paf_loss #paf
    ydot[5] = laao_catalyzed_biosynthesis - paca_external_efflux - paca_int_dil #paca_int 
    ydot[6] = paca_external_efflux #paca_ext
    ydot[7] = prom1_div - prom1_dil - pap_mrna_txn #promoter1
    ydot[8] = pap_mrna_txn - pap_mrna_dil - pap_mrna_deg - papA_tsn - papB_tsn - papC_tsn - pap_ared_folding #pap_mrna
    ydot[9] = pap_ared_folding - pap_ared_folded_dil - pap_ared_folded_deg - pap_ared_cleaving - papA_ared_folded_tsn - papB_ared_folded_tsn - papC_ared_folded_tsn #pap_ared_folded
    ydot[10] = pap_ared_cleaving - pap_ared_cleaved_dil - pap_ared_cleaved_deg - papA_ared_cleaved_tsn - papB_ared_cleaved_tsn - papC_ared_cleaved_tsn #pap_ared_cleaved
    ydot[11] = papA_tsn - papA_folding + papA_ared_cleaved_tsn + papA_ared_folded_tsn - papA_uf_dil - papA_uf_deg #papA_uf
    ydot[12] = papA_folding - papA_dil - papA_deg #papA
    ydot[13] = papB_tsn - papB_folding + papB_ared_cleaved_tsn + papB_ared_folded_tsn - papB_uf_dil - papB_uf_deg #papB_uf
    ydot[14] = papB_folding - papB_dil - papB_deg #papB
    ydot[15] = papC_tsn - papC_folding + papC_ared_cleaved_tsn + papC_ared_folded_tsn - papC_uf_dil - papC_uf_deg #papC_uf
    ydot[16] = papC_folding - papC_dil - papC_deg #papC 
    ydot[17] = deaminase_biosynthesis - deam_dil #deaminase
    ydot[18] = prom2_div - prom2_dil - laao_mrna_txn #promoter2
    ydot[19] = laao_mrna_txn - laao_mrna_deg - laao_mrna_dil - laao_tsn - laao_ared_folding #laao_mrna
    ydot[20] = laao_ared_folding - laao_ared_folded_dil - laao_ared_folded_deg - laao_ared_cleaving - laao_ared_folded_tsn #laao_ared_folded 
    ydot[21] = laao_ared_cleaving - laao_ared_cleaved_dil - laao_ared_cleaved_deg - laao_ared_cleaved_tsn #laao_ared_cleaved
    ydot[22] = laao_tsn - laao_folding + laao_ared_cleaved_tsn + laao_ared_folded_tsn - laao_uf_dil - laao_uf_deg #laao_uf
    ydot[23] = laao_folding - laao_dil - laao_deg #laao
    ydot[24] = prom3_div - prom3_dil - eff_mrna_txn #promoter3
    ydot[23] = eff_mrna_txn - eff_mrna_deg - eff_mrna_dil - eff_tsn - eff_ared_folding #eff_mrna
    ydot[23] = eff_ared_folding - eff_ared_folded_dil - eff_ared_folded_deg - eff_ared_cleaving - eff_ared_folded_tsn #eff_ared_folded
    ydot[23] = eff_ared_cleaving - eff_ared_cleaved_dil - eff_ared_cleaved_deg - eff_ared_cleaved_tsn #eff_ared_cleaved
    ydot[23] = eff_tsn - eff_folding + eff_ared_cleaved_tsn + eff_ared_folded_tsn - eff_uf_dil - eff_uf_deg#eff_uf
    ydot[23] = eff_folding - eff_dil - eff_deg #eff

In [ ]:
def p_aminostyrene(t, y, ydot, params):
    #Parse input parameters
    chorismate, pa1, pa2, pa3, paf, paca_int, paca_ext, promoter1, pap_mrna, papA_uf, papA, papB_uf, papB, papC_uf, papC, deaminase, promoter2, laao_mrna, laao_uf, laao, promoter3, eff_mrna, eff_uf, eff = y
    architecture, thetas, ks = params
    paf_arch, paca_arch = architecture
    theta_prom1, theta_prom2, theta_prom3 = thetas
    k_papA, k_papB, k_papC, k_prom2, k_prom3 = ks
    n = 2 #Fix n based on dimerization

    #Cellular constants
    chorismate_production_rate = 1100. #range [2E2, 2E3]
    deaminase_production_rate =  1E1 #range [1E0, 1E2]
    toxicity_factor = 1. 
    dilution_rate = 5.79E-4
    dna_duplication_rate = 5.78E-4
    avogadro = 6.0221408e+23
    cell_volume = 2.5E-15

    #Enzyme kinetic parameters
    enzyme_kcat = 5E0 #range [5E-1, 5E1]
    enzyme_km = 1E-6 #range [1E-7, 1E-5]
    papA_kcat = enzyme_kcat
    papA_km = enzyme_km
    papB_kcat = enzyme_kcat
    papB_km = enzyme_km
    papC_kcat = enzyme_kcat
    papC_km = enzyme_km
    laao_kcat = enzyme_kcat
    laao_km = enzyme_km
    deaminase_kcat = enzyme_kcat
    deaminase_km = enzyme_km
    efflux_rate = 275. #range [5E1, 5E2]

    #Kinetic pathway
    chorismate_biosynthesis = chorismate_production_rate * toxicity_factor
    deaminase_biosynthesis = deaminase_production_rate * toxicity_factor
    papA_catalyzed_biosynthesis = papA_kcat * papA * ((chorismate / avogadro) / cell_volume) / (papA_km + ((chorismate / avogadro) / cell_volume)) * toxicity_factor
    papB_catalyzed_biosynthesis = papB_kcat * ((pa1 / avogadro) / cell_volume) / (papB_km + ((pa1 / avogadro) / cell_volume)) * papB * toxicity_factor
    papC_catalyzed_biosynthesis = papC_km * papC * ((pa2 / avogadro) / cell_volume) / (papC_km + ((pa2 / avogadro) / cell_volume)) * toxicity_factor
    deaminase_catalyzed_biosynthesis = deaminase_kcat * deaminase * ((pa3 / avogadro) / cell_volume) / (deaminase_km + ((pa3 / avogadro) / cell_volume)) * toxicity_factor
    laao_catalyzed_biosynthesis = laao_kcat * laao * ((paf / avogadro) / cell_volume) / (laao_km + ((paf / avogadro) / cell_volume)) * toxicity_factor
    paca_external_efflux = eff * ((paca_int / avogadro) / cell_volume) * efflux_rate * toxicity_factor


    ydot[0] = chorismate_biosynthesis - papA_catalyzed_biosynthesis - chorismate*dilution_rate #chorismate
    ydot[1] = papA_catalyzed_biosynthesis - papB_catalyzed_biosynthesis - pa1*dilution_rate #pa1
    ydot[2] = papB_catalyzed_biosynthesis - papC_catalyzed_biosynthesis - pa2*dilution_rate #pa2
    ydot[3] = papC_catalyzed_biosynthesis - deaminase_catalyzed_biosynthesis - pa3*dilution_rate #pa3
    ydot[4] = deaminase_catalyzed_biosynthesis -  laao_catalyzed_biosynthesis - paf*dilution_rate - paf*1.40E-5 #range [1.4E-6, 1.4E-4] #paf
    ydot[5] = laao_catalyzed_biosynthesis - paca_external_efflux - paca_int*dilution_rate #paca_int 
    ydot[6] = paca_external_efflux #paca_ext

    ### IN PROGRESS ### - may need to split pap MRNA into 3 components to fit ks
    ydot[7] = promoter1*dna_duplication_rate - promoter1*dilution_rate - pap_mrna_txn #promoter1
    ydot[8] = pap_mrna_txn - pap_mrna * dilution_rate - pap_mrna_deg - papA_tsn - papB_tsn - papC_tsn - pap_ared_folding #pap_mrna





In [ ]:
def glucaric_acid(t, y, ydot, params):

    g6p, f6p, mi, ino1, miox, j1, j2 = y

    A, W = params 

    n_ino1, theta_ino1, k_ino1 = W[0]
    n_miox, theta_miox, k_miox = W[1]

    v_pgi = reversible_michaelismenten(g6p, f6p, vm_pgi, keq_pgi, km_pgi_g6p, km_pgi_f6p)
    v_zwf = michaelismenten(g6p, vm_zwf, km_zwf_g6p)
    v_pfk = hilleqn(f6p, vm_pfk, n_pfk, km_pfk_f6p)
    v_ino1 = ino1 * michaelismenten(g6p, vm_ino1, km_ino1_g6p)
    v_tm = michaelismenten(mi, vm_t_mi, km_t_mi)
    v_miox = miox * michaelismenten_substrateactivation(mi, vm_miox, km_miox_mi, a_miox, ka_miox_mi)

    u_ino1_mi = np.sum(A[0]*np.array([activation(mi, k_ino1, theta_ino1, n_ino1), repression(mi, k_ino1, theta_ino1, n_ino1), k_ino1]))
    u_miox_mi = np.sum(A[1]*np.array([activation(mi, k_miox, theta_miox, n_miox), repression(mi, k_miox, theta_miox, n_miox), k_miox]))

    ydot[0] = v_pts - v_zwf - v_pgi - lam*g6p
    ydot[1] = v_pgi + 0.5*v_zwf - v_pfk - lam*f6p
    ydot[2] = v_ino1 - v_tm - v_miox - lam*mi
    ydot[3] = u_ino1_mi  - lam*ino1
    ydot[4] = u_miox_mi - lam*miox
    ydot[5] = (v_pts - v_miox)**2
    ydot[6] = u_ino1_mi + u_miox_mi